In [3]:
from rasterio.warp import reproject, Resampling, calculate_default_transform
import rasterio
def reproj_match(infile, match, outfile):
    with rasterio.open(infile) as src:
        src_transform = src.transform
        
        with rasterio.open(match) as match:
            dst_crs = match.crs
            
            dst_transform, dst_width, dst_height = calculate_default_transform(
                src.crs,    
                dst_crs,    
                match.width,   
                match.height,  
                *match.bounds,
            )

        dst_kwargs = src.meta.copy()
        dst_kwargs.update({"crs": dst_crs,
                           "transform": dst_transform,
                           "width": dst_width,
                           "height": dst_height,
                           "nodata": 0})
        print("Coregistered to shape:", dst_height,dst_width,'\n Affine',dst_transform)
        with rasterio.open(outfile, "w", **dst_kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

In [2]:
LS = r"C:\Users\rishi\OneDrive\Desktop\Rishika_data\resampled_sentinel_1.tif"
precip = r"C:\Users\rishi\OneDrive\Desktop\Rishika_data\20230930_mhowroad_planet_data.tif"

reproj_match(infile = LS, 
             match= precip,
             outfile = r"C:\Users\rishi\OneDrive\Desktop\Rishika_data\co registered_sentinel_1.tif")

Coregistered to shape: 428 1011 
 Affine | 3.00, 0.00, 590700.00|
| 0.00,-3.00, 2492079.00|
| 0.00, 0.00, 1.00|
